In [169]:
import pandas as pd
import numpy as np

In [183]:
df = pd.read_csv("test_match_logs_with_elos_20250606.csv")
df_odds_test = pd.read_csv("df_test_cleaned.csv")

In [198]:
len(df)

59461

In [185]:
df_odds_test.columns[:60]

Index(['date', 'round', 'game_started', 'position', 'passes', 'player_name',
       'league', 'is_post_transfer_match', 'opponent_strength',
       'opponent_defensive_strength', 'team_offensive_strength_last_3',
       'opponent_defensive_strength_last_3', 'team_offensive_strength_last_5',
       'opponent_defensive_strength_last_5', 'is_home_next_game',
       'target_scored_next_game', 'next_game_date', 'strength_diff_next_game',
       'team_offensive_strength_next_game', 'team_strength_next_game',
       'team_elo_next_game', 'team_offensive_strength_last_3_next_game',
       'team_offensive_strength_last_5_next_game',
       'opponent_defensive_strength_next_game', 'opponent_strength_next_game',
       'opponent_elo_next_game',
       'opponent_defensive_strength_last_3_next_game',
       'opponent_defensive_strength_last_5_next_game',
       'offensive_defensive_multiply_next_game',
       'offensive_defensive_multiply_last_3_next_game',
       'offensive_defensive_multiply_last

In [199]:
def calculate_elo_delta_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to calculate the elo delta feature. This feature is the difference between
    the current team elo for post-transfer matches and the last team elo from the club elo
    prior to the transfer.
    """
    
    df = df.copy()
    df['delta_elo_post_transfer'] = np.nan
    
    # Group by transfer_id to process each transfer separately
    for transfer_id, transfer_group in df.groupby('transfer_id'):
        transfer_group_sorted = transfer_group.sort_values('date')
        pre_transfer_rows = transfer_group_sorted[
            (transfer_group_sorted['is_pre_transfer']) & 
            (transfer_group_sorted['team_elo_from_club_elo'].notna())
        ]
        if len(pre_transfer_rows) > 0:
            # Get the last pre-transfer elo value (chronologically last)
            last_pre_transfer_elo = pre_transfer_rows['team_elo_from_club_elo'].iloc[-1]
            # Calculate delta for post-transfer rows with non-null elo values
            post_transfer_mask = (
                (df['transfer_id'] == transfer_id) & 
                (df['is_post_transfer']) &
                (df['team_elo_from_club_elo'].notna())
            )
            df.loc[post_transfer_mask, 'delta_elo_post_transfer'] = (
                df.loc[post_transfer_mask, 'team_elo_from_club_elo'] - last_pre_transfer_elo
            )
    return df

In [200]:
df = calculate_elo_delta_feature(df)

In [188]:
def add_team_delta_features(
    df: pd.DataFrame,
    team_stats_dfs: dict,
    feature_df_mapping: dict
) -> pd.DataFrame:
    """
    Add team delta features to the test match logs dataframe.
    
    This function calculates the difference between post-transfer team stats 
    and pre-transfer team stats for the same season preceding the transfer.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The test match logs dataframe with transfer information
    team_stats_dfs : dict
        Dictionary mapping stat type names to their corresponding dataframes
        e.g., {'shooting': shooting_df, 'standard': standard_df}
    feature_df_mapping : dict
        Dictionary mapping feature names to the dataframe they should be sourced from
        e.g., {'goals': 'standard', 'shots': 'shooting', 'assists': 'standard'}
        
    Returns:
    --------
    pd.DataFrame
        The original dataframe with added delta feature columns
    """
    
    result_df = df.copy()
    for feature in feature_df_mapping.keys():
        result_df[f'{feature}_delta'] = pd.NA
    
    for _, transfer_group in result_df.groupby('transfer_id'):
        
        pre_transfer_matches = transfer_group[transfer_group['is_pre_transfer'] == True]
        post_transfer_matches = transfer_group[transfer_group['is_post_transfer'] == True]
        
        # Skip if we don't have both pre and post transfer matches
        if pre_transfer_matches.empty or post_transfer_matches.empty:
            continue
        pre_transfer_sample = pre_transfer_matches.iloc[0]
        post_transfer_sample = post_transfer_matches.iloc[0]
        reference_season = pre_transfer_sample['season']
        pre_transfer_league = pre_transfer_sample['league']
        pre_transfer_team = pre_transfer_sample['team']
        post_transfer_league = post_transfer_sample['league']
        post_transfer_team = post_transfer_sample['team']
        
        delta_values = {}
        
        for feature, df_name in feature_df_mapping.items():
            if df_name not in team_stats_dfs:
                print(f"Warning: Dataframe '{df_name}' not found in team_stats_dfs")
                delta_values[f'{feature}_delta'] = pd.NA
                continue
                
            stats_df = team_stats_dfs[df_name]
            
            if feature not in stats_df.columns:
                print(f"Warning: Feature '{feature}' not found in '{df_name}' dataframe")
                delta_values[f'{feature}_delta'] = pd.NA
                continue
            
            pre_team_mask = (
                (stats_df['league'] == pre_transfer_league) &
                (stats_df['season'] == reference_season) &
                (stats_df['team_name'] == pre_transfer_team)
            )
            pre_team_matches = stats_df[pre_team_mask]
            
            post_team_mask = (
                (stats_df['league'] == post_transfer_league) &
                (stats_df['season'] == reference_season) &
                (stats_df['team_name'] == post_transfer_team)
            )
            post_team_matches = stats_df[post_team_mask]
            
            # Calculate delta (post - pre)
            if not pre_team_matches.empty and not post_team_matches.empty:
                pre_team_stat = pre_team_matches[feature].iloc[0]
                post_team_stat = post_team_matches[feature].iloc[0]
                delta_values[f'{feature}_delta'] = post_team_stat - pre_team_stat
            else:
                delta_values[f'{feature}_delta'] = pd.NA
        
        # Apply the calculated deltas to all rows in this transfer group
        for delta_feature, delta_value in delta_values.items():
            result_df.loc[transfer_group.index, delta_feature] = delta_value
    
    return result_df

In [189]:
standard_df = pd.read_csv("team_stats_standard_20250530.csv")
shooting_df = pd.read_csv("team_stats_shooting_20250530.csv")

In [175]:
team_stats_dfs = {
    'standard': standard_df,
    'shooting': shooting_df
}

feature_df_mapping = {
    'goals_per90': 'standard', # Use goals from standard stats since it's more complete
    'goals_assists_per90': 'standard',
    'xg_per90' : 'standard',
    'shots_per90': 'shooting',
    'shots_on_target_per90': 'shooting',
    'average_shot_distance' : 'shooting',
    "xg_assist_per90" : "standard",
    "possession" : "standard"   
}

In [190]:
result_df = add_team_delta_features(df, team_stats_dfs, feature_df_mapping)

In [111]:
result_df.columns

Index(['date', 'dayofweek', 'round', 'venue', 'result', 'team', 'opponent',
       'game_started', 'position', 'minutes', 'goals', 'assists', 'pens_made',
       'pens_att', 'shots', 'shots_on_target', 'cards_yellow', 'cards_red',
       'touches', 'tackles', 'interceptions', 'blocks', 'xg', 'npxg',
       'xg_assist', 'sca', 'gca', 'passes_completed', 'passes', 'passes_pct',
       'progressive_passes', 'carries', 'progressive_carries', 'take_ons',
       'take_ons_won', 'player_name', 'player_id', 'stat_type', 'league',
       'is_mid_season_transfer', 'season', 'comp', 'fouls', 'fouled',
       'offsides', 'crosses', 'tackles_won', 'own_goals', 'pens_won',
       'pens_conceded', 'transfer_id', 'transfer_date', 'from_club', 'to_club',
       'is_post_transfer', 'is_pre_transfer', 'days_since_transfer',
       'match_number_after_transfer', 'team_elo_from_club_elo',
       'delta_elo_post_transfer', 'goals_per90_delta',
       'goals_assists_per90_delta', 'xg_per90_delta', 'shots_per

In [194]:
final_df = df_odds_test.merge(result_df[["date","player_name",'from_club', 'to_club','delta_elo_post_transfer', 'goals_per90_delta',
       'goals_assists_per90_delta', 'xg_per90_delta', 'shots_per90_delta','season',
       'shots_on_target_per90_delta', 'average_shot_distance_delta',"round","league",
       'xg_assist_per90_delta', 'possession_delta']], on=["date", "player_name","round","league"], how="left")

In [197]:
len(final_df[(final_df["delta_elo_post_transfer"].notna()) & (final_df["is_post_transfer_match"] == True)])

4282

In [ ]:
final_df[(final_df["delta_elo_post_transfer"].isna())  & (final_df["is_post_transfer_match"] == True)]["season"].value_counts()

season
2023-2024    8111
2022-2023    6927
2024-2025    6372
2021-2022    4145
Name: count, dtype: int64